.# Эксперимент 1: Оптимизация функции y(x) = sin(x) + e^x

В этом ноутбуке мы используем предобученную PABBO модель для оптимизации кастомной функции `y(x) = sin(x) + e^x`.

## Предварительные требования

Перед запуском этого эксперимента необходимо:
1. Предобучить модель PABBO на GP1D (см. `simple_experiment.ipynb`)
2. Убедиться, что чекпоинт существует: `../pabbo_method/results/evaluation/PABBO/PABBO_GP1D/ckpt.tar`

In [ ]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import os.path as osp

# Добавляем путь к модулю pabbo_method
sys.path.append('../pabbo_method')

print(f"PyTorch version: {torch.__version__}")
print(f"Current working directory: {os.getcwd()}")

## Проверка предобученной модели

In [ ]:
# Проверяем наличие чекпоинта
checkpoint_path = "../pabbo_method/results/evaluation/PABBO/PABBO_GP1D/ckpt.tar"

if osp.exists(checkpoint_path):
    print(f"✅ Чекпоинт найден: {checkpoint_path}")
    ckpt = torch.load(checkpoint_path, map_location='cpu')
    print(f"Шаг обучения: {ckpt.get('step', 'N/A')}")
else:
    print(f"❌ Чекпоинт не найден!")
    print("Сначала запустите предобучение из simple_experiment.ipynb")

## Визуализация целевой функции

Давайте сначала посмотрим, как выглядит функция `y(x) = sin(x) + e^x`

In [ ]:
# Импортируем нашу кастомную функцию
from data.function import sinexp1D

# Создаем сетку значений x
x_range = [-1, 1]  # Диапазон для оптимизации
x = torch.linspace(x_range[0], x_range[1], 1000).unsqueeze(-1)
y = sinexp1D(x, negate=False, add_dim=False)

# Визуализация
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(x.numpy(), y.numpy(), 'b-', linewidth=2)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Функция y(x) = sin(x) + e^x')
plt.grid(True)

# Для оптимизации мы максимизируем -y (convention в PABBO)
y_opt = sinexp1D(x, negate=True, add_dim=False)

plt.subplot(1, 2, 2)
plt.plot(x.numpy(), y_opt.numpy(), 'r-', linewidth=2)
plt.xlabel('x')
plt.ylabel('-y')
plt.title('Целевая функция для максимизации: -y(x)')
plt.grid(True)

plt.tight_layout()
plt.show()

# Находим максимум
max_idx = torch.argmax(y_opt)
x_max = x[max_idx].item()
y_max = y_opt[max_idx].item()

print(f"\nГлобальный максимум (приблизительно):")
print(f"x* ≈ {x_max:.4f}")
print(f"y* ≈ {y_max:.4f}")
print(f"\nПримечание: PABBO будет пытаться найти этот максимум через парные сравнения")

## Запуск оптимизации с помощью PABBO

Теперь используем предобученную модель для оптимизации нашей функции.

### Параметры эксперимента:
- `experiment.model=PABBO` - использование модели PABBO
- `experiment.expid=PABBO_GP1D` - ID предобученного эксперимента
- `data.name=sinexp1D` - наша кастомная функция
- `data.d_x=1` - одномерная оптимизация
- `data.x_range="[[-1,1]]"` - диапазон поиска
- `eval.eval_max_T=30` - количество итераций оптимизации

In [ ]:
# Команда для запуска оптимизации
eval_command = """
cd ../pabbo_method && python evaluate_continuous.py --config-name=evaluate \
    experiment.model=PABBO \
    experiment.expid=PABBO_GP1D \
    experiment.device=cpu \
    eval.eval_max_T=30 \
    eval.eval_num_query_points=256 \
    eval.num_parallel=1 \
    data.name=sinexp1D \
    data.d_x=1 \
    data.x_range=\"[[-1,1]]\" \
    data.Xopt=\"[[{xopt}]]\" \
    data.yopt=\"[[{yopt}]]\" \
    experiment.wandb=false \
    eval.num_seeds=5 \
    eval.num_datasets=1
""".format(xopt=x_max, yopt=y_max)

print("Команда для запуска оптимизации:")
print(eval_command)
print("\n⚠️ Параметры:")
print(f"  - eval.eval_max_T=30 (30 итераций оптимизации)")
print(f"  - eval.eval_num_query_points=256 (256 точек в пространстве поиска)")
print(f"  - eval.num_parallel=1 (последовательный поиск)")
print(f"  - eval.num_seeds=5 (5 случайных запусков для статистики)")
print(f"  - data.Xopt и data.yopt заданы на основе численного анализа")

In [ ]:
# Раскомментируйте для запуска оптимизации
# ВНИМАНИЕ: Это может занять время!

# import subprocess
# result = subprocess.run(
#     eval_command,
#     shell=True,
#     capture_output=True,
#     text=True
# )
# print(result.stdout)
# if result.stderr:
#     print("STDERR:", result.stderr)

## Анализ результатов

После завершения оптимизации результаты сохраняются в:
`results/evaluation/sinexp1D/PABBO/PABBO_GP1D/`

In [ ]:
# Путь к результатам
result_dir = "../pabbo_method/results/evaluation/sinexp1D/PABBO/PABBO_GP1D"

# Проверяем наличие результатов
if osp.exists(result_dir):
    print(f"✅ Результаты найдены: {result_dir}")
    
    # Список всех файлов результатов
    import os
    files = os.listdir(result_dir)
    print(f"\nНайденные файлы:")
    for f in files:
        print(f"  - {f}")
else:
    print(f"❌ Результаты не найдены")
    print("Сначала запустите оптимизацию")

In [ ]:
# Загрузка и визуализация результатов
def plot_optimization_results(result_dir):
    """
    Визуализация результатов оптимизации PABBO
    """
    try:
        # Загружаем метрики
        simple_regret = torch.load(f"{result_dir}/SIMPLE_REGRET_S256_B1.pt")
        immediate_regret = torch.load(f"{result_dir}/IMMEDIATE_REGRET_S256_B1.pt")
        entropy = torch.load(f"{result_dir}/ENTROPY_S256_B1.pt")
        kt_cor = torch.load(f"{result_dir}/KT_COR_S256_B1.pt")
        
        print(f"Simple regret shape: {simple_regret.shape}")
        print(f"  - num_seeds: {simple_regret.shape[0]}")
        print(f"  - num_datasets: {simple_regret.shape[1]}")
        print(f"  - num_steps: {simple_regret.shape[2]}")
        
        # Создаем графики
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # 1. Simple Regret
        ax = axes[0, 0]
        mean_regret = simple_regret.mean(dim=(0, 1))
        std_regret = simple_regret.std(dim=(0, 1))
        steps = range(len(mean_regret))
        
        ax.plot(steps, mean_regret, 'b-', linewidth=2, label='Mean')
        ax.fill_between(steps, 
                        mean_regret - std_regret, 
                        mean_regret + std_regret, 
                        alpha=0.3, color='b')
        ax.set_xlabel('Optimization Step')
        ax.set_ylabel('Simple Regret')
        ax.set_title('Simple Regret vs Step')
        ax.legend()
        ax.grid(True)
        
        # 2. Immediate Regret
        ax = axes[0, 1]
        mean_imm_regret = immediate_regret.mean(dim=(0, 1))
        std_imm_regret = immediate_regret.std(dim=(0, 1))
        
        ax.plot(steps, mean_imm_regret, 'r-', linewidth=2, label='Mean')
        ax.fill_between(steps, 
                        mean_imm_regret - std_imm_regret, 
                        mean_imm_regret + std_imm_regret, 
                        alpha=0.3, color='r')
        ax.set_xlabel('Optimization Step')
        ax.set_ylabel('Immediate Regret')
        ax.set_title('Immediate Regret vs Step')
        ax.legend()
        ax.grid(True)
        
        # 3. Entropy
        ax = axes[1, 0]
        mean_entropy = entropy.mean(dim=(0, 1))
        std_entropy = entropy.std(dim=(0, 1))
        steps_entropy = range(len(mean_entropy))
        
        ax.plot(steps_entropy, mean_entropy, 'g-', linewidth=2, label='Mean')
        ax.fill_between(steps_entropy, 
                        mean_entropy - std_entropy, 
                        mean_entropy + std_entropy, 
                        alpha=0.3, color='g')
        ax.set_xlabel('Optimization Step')
        ax.set_ylabel('Entropy')
        ax.set_title('Policy Entropy vs Step')
        ax.legend()
        ax.grid(True)
        
        # 4. Kendall-Tau Correlation
        ax = axes[1, 1]
        mean_kt = kt_cor.mean(dim=(0, 1))
        std_kt = kt_cor.std(dim=(0, 1))
        
        ax.plot(steps_entropy, mean_kt, 'm-', linewidth=2, label='Mean')
        ax.fill_between(steps_entropy, 
                        mean_kt - std_kt, 
                        mean_kt + std_kt, 
                        alpha=0.3, color='m')
        ax.set_xlabel('Optimization Step')
        ax.set_ylabel('Kendall-Tau Correlation')
        ax.set_title('Ranking Quality vs Step')
        ax.legend()
        ax.grid(True)
        
        plt.tight_layout()
        plt.show()
        
        # Выводим финальную статистику
        print(f"\n📊 Финальные результаты:")
        print(f"  Simple Regret: {mean_regret[-1]:.4f} ± {std_regret[-1]:.4f}")
        print(f"  Kendall-Tau Correlation: {mean_kt[-1]:.4f} ± {std_kt[-1]:.4f}")
        print(f"  Entropy: {mean_entropy[-1]:.4f} ± {std_entropy[-1]:.4f}")
        
    except Exception as e:
        print(f"Ошибка при загрузке результатов: {e}")
        import traceback
        traceback.print_exc()

# Раскомментируйте для визуализации
# plot_optimization_results(result_dir)

## Интерпретация результатов

**Simple Regret** показывает разницу между оптимальным значением функции и лучшим найденным значением. Чем меньше Simple Regret, тем ближе мы к оптимуму.

**Immediate Regret** показывает качество последнего запроса.

**Kendall-Tau Correlation** измеряет качество ранжирования: насколько хорошо модель понимает относительный порядок значений функции.

**Entropy** показывает уверенность политики: низкая энтропия означает, что модель уверена в выборе следующей точки.

## Выводы и следующие шаги

После успешного запуска этого эксперимента:

1. Проанализируйте, насколько быстро PABBO находит оптимум функции sin(x) + e^x
2. Сравните с другими методами оптимизации (можно добавить baseline методы)
3. Попробуйте изменить параметры:
   - Диапазон x (например, [-2, 2])
   - Количество итераций (eval.eval_max_T)
   - Размер пространства поиска (eval.eval_num_query_points)

Далее можно перейти к Эксперименту 2: оптимизация количества тем LDA.